In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
df.shape

(90, 2)

In [5]:
def tokenize(text):
    text = text.lower()
    text = text.replace('?', ' . ')
    text = text.replace("'", ' . ')
    return text.split()

In [6]:
tokenize("Who is swapnil")

['who', 'is', 'swapnil']

In [7]:
vocab = {'<UNK>': 0}

In [11]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)

In [12]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [14]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [15]:
text_to_indices("Who is swapnil", vocab)

[11, 2, 0]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [21]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [23]:
dataset = QADataset(df, vocab)

In [24]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6, 7]), tensor([8]))

In [25]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [26]:
import torch.nn as nn

In [41]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [42]:
learning_rate = 0.001
epochs = 20

In [43]:
model = SimpleRNN(len(vocab))

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [45]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 531.597882
Epoch: 2, Loss: 433.445010
Epoch: 3, Loss: 382.415935
Epoch: 4, Loss: 346.645191
Epoch: 5, Loss: 311.028209
Epoch: 6, Loss: 273.862490
Epoch: 7, Loss: 238.958238
Epoch: 8, Loss: 202.642979
Epoch: 9, Loss: 170.774941
Epoch: 10, Loss: 141.714880
Epoch: 11, Loss: 115.988404
Epoch: 12, Loss: 94.598046
Epoch: 13, Loss: 76.552787
Epoch: 14, Loss: 62.237908
Epoch: 15, Loss: 49.826891
Epoch: 16, Loss: 40.898136
Epoch: 17, Loss: 33.734242
Epoch: 18, Loss: 27.926858
Epoch: 19, Loss: 23.494267
Epoch: 20, Loss: 20.026827


In [68]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    return "I dont know"
  
  return list(vocab.keys())[index]

In [69]:
predict(model, "What is the largest planet in our solar system?")

'jupiter'

In [70]:
predict(model, "Which is the smallest city in India")

'I dont know'